# 1-bus UCBlock and ThermalUnitBlock

In the following, we provide a quick example on how to add a simple optimization model with **SMS++**.
The problem below optimizes the dispatch of a single thermal generator of 100 kW to meet a constant load
over 24 hours in one bus.


A sample **SMS++** network can be created with the following code. After the Python imports, the code creates
a new SMS++ network `sn` with the **block file** format. The block file format is a text file that contains
only the model data in a structured way, with no solver information. The solver information is provided
in a separate configuration file.

In [ ]:
from pysmspp import SMSNetwork, Variable, Block, SMSFileType
import numpy as np

# Create an empty SMSNetwork with block file type
sn = SMSNetwork(file_type=SMSFileType.eBlockFile)

# For demonstration, we'll print out the network to confirm it is created.
sn

After an empty network is created, we can populate it by blocks. In particular, we need to add a first
inner block that describes the type of model to be optimized. Here, we add a `UCBlock` (Unit Commitment
Block) suitable for unit commitment problems (see [UCBlock SMS documentation](https://gitlab.com/smspp/ucblock)
for more details).We specify **24 time steps** (one day) and a **constant demand** of **50 kW** for each time
step.

The block is added to the network with:

In [ ]:
ucb = sn.add(
    "UCBlock",  # block type
    "Block_0",  # block name
    id="0",  # block id
    TimeHorizon=24,  # number of time steps
    NumberUnits=1,  # number of units
    NumberElectricalGenerators=1,  # number of electrical generators
    NumberNodes=1,  # number of nodes
    ActivePowerDemand=Variable(  # active power demand
        "ActivePowerDemand",
        "float",
        ("NumberNodes", "TimeHorizon"),
        np.full((1, 24), 50.0),  # constant demand of 50 kW
    ),
)

print("Added UCBlock with constant demand.")

ucb  # print the block to confirm it is created

In the unit commitment block above, no generator is yet added. To add a generator, we first create a
`ThermalUnitBlock`. Then we attach it to `sn.blocks["Block_0"]` using the `add_block` method. The snippet
below sets up a thermal generator with 100 kW maximum output.

In [ ]:
thermal_unit_block = Block().from_kwargs(
    block_type="ThermalUnitBlock",
    MinPower=Variable("MinPower", "float", (), 0.0),
    MaxPower=Variable("MaxPower", "float", (), 100.0),
    LinearTerm=Variable("LinearTerm", "float", (), 0.3),
    InitUpDownTime=Variable("InitUpDownTime", "int", (), 1),
)

# Add it to the existing UCBlock (Block_0)
sn.blocks["Block_0"].add_block("UnitBlock_0", block=thermal_unit_block)
print("ThermalUnitBlock added.")

Finally, we can optimize the network using a solver configuration file and specifying a temporary SMS++
file path. Here’s an example invocation:

In [ ]:
configfile = "../../test/test_data/configs/UCBlockSolver/uc_solverconfig.txt"  # path to solver config file
temporary_smspp_file = "./smspp_temp_file.nc"  # path to temporary SMS++ file
output_file = "./smspp_output.txt"  # path to the output file (optional)

result = sn.optimize(
    configfile,
    temporary_smspp_file,
    output_file,
)

print("Optimization finished.")

Basic results are stored in the `result` object. For instance, you can check the solver status and
the final objective value with:

In [ ]:
print("Status:", result.status)
print("Objective value:", result.objective_value)